In [1]:
import altair as alt
import numpy as np
import math
import polars as pl
import requests

from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.tile_providers import OSM, get_provider
from bokeh.io import output_notebook

tile_provider = get_provider(OSM)
output_notebook()

Loading BokehJS ...

In [2]:
positions = pl.read_csv("positions.csv")

In [3]:
# https://wiki.openstreetmap.org/wiki/Mercator#Python
MERCATOR_RADIUS = 6378137

positions = positions.filter(
    (pl.col("x") > -90) & (pl.col("x") < 90) & (pl.col("y") > -180) & (pl.col("y") < 180)
).sort(
    ["k", "timestamp"]
).with_columns([
    (pl.col("timestamp")+"Z").str.strptime(pl.Datetime, fmt="%+", strict=False),
    np.deg2rad(pl.col("x")).alias("x_rad"),
    np.deg2rad(pl.col("y")).alias("y_rad"),
]).with_columns([
    pl.internals.expr.ExprDateTimeNameSpace.seconds(pl.col("timestamp")-pl.col("timestamp").shift(1)).alias("time_diff_s"),
    ((pl.col("x_rad")/2 + math.pi/4).tan().log() * MERCATOR_RADIUS).alias("y_mercator"),
    (pl.col("y_rad") * MERCATOR_RADIUS).alias("x_mercator"),
])

In [4]:
# https://stackoverflow.com/questions/365826/calculate-distance-between-2-gps-coordinates
positions['distance_km'] = positions.select([
    pl.col("x_rad").alias("x"),
    pl.col("y_rad").alias("y"),
    pl.col("x_rad").shift(1).alias("prev_x"),
    pl.col("y_rad").shift(1).alias("prev_y")
]).with_columns([
    (pl.col("x") - pl.col("prev_x")).alias("d_lat"),
    (pl.col("y") - pl.col("prev_y")).alias("d_lon"),
]).with_columns([
    ((pl.col("d_lat") / 2).sin().pow(2) + (pl.col("d_lon") / 2).sin().pow(2) * pl.col("prev_x").cos() * pl.col("x").cos()).alias("a")
]).with_columns([
    ((pl.col("a").sqrt() / (1-pl.col("a")).sqrt()).arctan() * 2 * 6373).alias("distance_km")
])['distance_km']

In [5]:
positions = positions.with_column(
    pl.when(pl.col("k").is_first())
    .then(None)
    .otherwise(pl.col("time_diff_s")).alias("time_diff_s")
).with_column(
    pl.when(pl.col("k").is_first())
    .then(None)
    .otherwise(pl.col("distance_km")).alias("distance_km")
).with_column(
    (pl.col("distance_km") / pl.col("time_diff_s") * 3600).alias("speed_km_h")
)

In [6]:
# positions.groupby('k').agg(pl.col("k").count()).sort("k_count")
test = positions.filter(pl.col("k") == 19713067)

In [7]:
# https://docs.bokeh.org/en/latest/docs/reference/models/formatters.html?highlight=datetimetickformatter#bokeh.models.DatetimeTickFormatter
p = figure(title="Simple line example", x_axis_type = "datetime", x_axis_label='x', y_axis_label='y')
p.line(
    test["timestamp"],
    test["speed_km_h"],
    legend_label="Temp.",
    line_width=2
)
show(p)

In [8]:
p = figure(
    x_range=(1875733.419866658, 1914695.2416443035), 
    y_range=(6621293.72274016, 6674532.798473069),
    x_axis_type="mercator",
    y_axis_type="mercator"
)
p.add_tile(tile_provider)


source = ColumnDataSource(
    data=test.select([
        pl.col("x_mercator"),
        pl.col("y_mercator"),
        pl.col("speed_km_h"),
    ]).to_dict(as_series=False)
)


p.circle(x="x_mercator", y="y_mercator", size=5, fill_color="blue", fill_alpha=0.8, source=source)

show(p)
# https://docs.bokeh.org/en/latest/docs/user_guide/geo.html?highlight=geojson
# TODO

In [9]:
positions.groupby('name').agg([
    pl.col("k").n_unique(),
    (pl.col("distance_km").sum() / pl.col("time_diff_s").sum() * 3600).alias("speed_km_h_avg")
]).sort("speed_km_h_avg")

name,k_n_unique,speed_km_h_avg
str,u32,f64
"""700""",198,10.704677
"""8""",123,10.786523
"""147""",9,10.930407
"""23""",106,11.373344
"""4""",123,11.601943
"""15""",121,11.922381
"""33""",113,12.0247
"""6""",112,12.262644
"""7""",138,12.280575
